In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = '5g'
spark = SparkSession.builder.appName("taxi-fare-perdiction")\
                .config("spark.executor.memory", MAX_MEMORY)\
                .config("spark.driver.memory", MAX_MEMORY)\
                .getOrCreate()

In [3]:
trip_files = "/Users/gimhyeonjeong/data-engineering/01-spark/data/trip_csv/*"

In [4]:
trips_df = spark.read.csv(f"file:///{trip_files}", inferSchema=True, header= True)

In [5]:
trips_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [6]:
trips_df.createOrReplaceTempView("trips")

In [10]:
query = """
SELECT 
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")

In [11]:
data_df.show()

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|            1.0|               142|                 43|          2.1|          0|     Friday|        11.8|
|            1.0|               238|                151|          0.2|          0|     Friday|         4.3|
|            1.0|               132|                165|         14.7|          0|     Friday|       51.95|
|            0.0|               138|                132|         10.6|          0|     Friday|       36.35|
|            1.0|                68|                 33|         4.94|          0|     Friday|       24.36|
|            1.0|               224|                 68|          1.6|          0|     Friday|       14.15|
|            1.0|           

In [12]:
data_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [13]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=1)

In [17]:
data_dir = "/Users/gimhyeonjeong/data-engineering/01-spark/data/"

In [18]:
train_df.write.format("parquet").save(f"{data_dir}/train/")
test_df.write.format("parquet").save(f"{data_dir}/test/")

In [20]:
train_df = spark.read.parquet(f"{data_dir}/train/")
test_df = spark.read.parquet(f"{data_dir}/test/")

In [21]:
train_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [25]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

cat_feats = [
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week"
]

stages = []

for c in cat_feats:
    cat_indexer = StringIndexer(inputCol= c, outputCol= c + "_idx").setHandleInvalid("keep")
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()], outputCols=[c + "_onehot"])
    stages += [cat_indexer, onehot_encoder]

In [26]:
stages

[StringIndexer_875b27c36a4b,
 OneHotEncoder_854d6964db75,
 StringIndexer_5fff85e80ee1,
 OneHotEncoder_b710ff7015e8,
 StringIndexer_afe3f092d53c,
 OneHotEncoder_bb0e7dccbcb9]

In [29]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

num_feats = [
    "passenger_count",
    "trip_distance",
    "pickup_time"
]

for n in num_feats:
    num_assembler = VectorAssembler(inputCols=[n], outputCol= n + "_vector")
    num_scaler = StandardScaler(inputCol = num_assembler.getOutputCol(), outputCol = n + "_scaled")
    stages += [num_assembler, num_scaler]

In [30]:
stages

[StringIndexer_875b27c36a4b,
 OneHotEncoder_854d6964db75,
 StringIndexer_5fff85e80ee1,
 OneHotEncoder_b710ff7015e8,
 StringIndexer_afe3f092d53c,
 OneHotEncoder_bb0e7dccbcb9,
 VectorAssembler_0088d577bc96,
 StandardScaler_f05a32107fea,
 VectorAssembler_afeee0f10b3d,
 StandardScaler_e9ab3cd1b821,
 VectorAssembler_b1a1cfdd1e22,
 StandardScaler_47282d9fd7a7]

In [39]:
assembler_inputs = [c + "_onehot" for c in cat_feats] + [n + "_scaled" for n in num_feats]
assembler = VectorAssembler(inputCols = assembler_inputs, outputCol= "feature_vector")
stages += [assembler]

In [41]:
stages

[StringIndexer_875b27c36a4b,
 OneHotEncoder_854d6964db75,
 StringIndexer_5fff85e80ee1,
 OneHotEncoder_b710ff7015e8,
 StringIndexer_afe3f092d53c,
 OneHotEncoder_bb0e7dccbcb9,
 VectorAssembler_0088d577bc96,
 StandardScaler_f05a32107fea,
 VectorAssembler_afeee0f10b3d,
 StandardScaler_e9ab3cd1b821,
 VectorAssembler_b1a1cfdd1e22,
 StandardScaler_47282d9fd7a7,
 VectorAssembler_c0f1e34a5cc1]

In [40]:
from pyspark.ml import Pipeline

transform_stages = stages 
pipeline = Pipeline(stages = transform_stages)
fitted_transformer = pipeline.fit(train_df)

In [42]:
vtrain_df = fitted_transformer.transform(train_df)

In [43]:
from pyspark.ml.regression import LinearRegression 

lr = LinearRegression(
    maxIter = 5,
    solver = "normal",
    labelCol = "total_amount",
    featuresCol = "feature_vector"
)

In [44]:
vtrain_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vector: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vector: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vector: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (null

In [45]:
model = lr.fit(vtrain_df)

In [46]:
vtest_df = fitted_transformer.transform(test_df)

In [48]:
predictions = model.transform(vtest_df)

In [49]:
predictions.cache()

DataFrame[passenger_count: double, pickup_location_id: int, dropoff_location_id: int, trip_distance: double, pickup_time: int, day_of_week: string, total_amount: double, pickup_location_id_idx: double, pickup_location_id_onehot: vector, dropoff_location_id_idx: double, dropoff_location_id_onehot: vector, day_of_week_idx: double, day_of_week_onehot: vector, passenger_count_vector: vector, passenger_count_scaled: vector, trip_distance_vector: vector, trip_distance_scaled: vector, pickup_time_vector: vector, pickup_time_scaled: vector, feature_vector: vector, prediction: double]

In [50]:
predictions.show()

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+-----------------------+--------------------------+---------------+------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|pickup_location_id_idx|pickup_location_id_onehot|dropoff_location_id_idx|dropoff_location_id_onehot|day_of_week_idx|day_of_week_onehot|passenger_count_vector|passenger_count_scaled|trip_distance_vector|trip_distance_scaled|pickup_time_vector|  pickup_time_scaled|      feature_vector|        prediction|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+----------------------

In [51]:
predictions.select(["trip_distance", "day_of_week", "total_amount", "prediction"]).show()

+-------------+-----------+------------+------------------+
|trip_distance|day_of_week|total_amount|        prediction|
+-------------+-----------+------------+------------------+
|          0.7|   Saturday|       12.35|12.502162410916714|
|          3.1|    Tuesday|        18.0| 17.78264885094742|
|          2.1|   Saturday|       15.35|16.787068747705387|
|          1.7|   Saturday|        13.3|14.298237840437643|
|          4.1|     Friday|        21.3|20.893785642932762|
|          1.4|     Friday|         8.3|11.950093031236984|
|          7.3|    Tuesday|        29.3|27.652166598371807|
|          0.7|  Wednesday|         5.8|  9.63923529763698|
|          5.0|  Wednesday|        24.3|21.248267279006047|
|          6.7|   Saturday|        29.8| 35.68814837777281|
|         16.8|     Friday|       82.37| 71.71520136691043|
|         29.3|     Monday|        80.8|102.87405109139928|
|          4.1|     Friday|        20.8| 22.31762454838676|
|          0.1|  Wednesday|        55.3|

In [53]:
model.summary.rootMeanSquaredError

5.856854592689126

In [55]:
model.summary.r2

0.7964059503830232